# Riak

In [1]:
import riak
from pprintpp import pprint as pp
import json
import pandas as pd

import uuid
import hashlib

## Lectura de información en Pandas

Partimos del dataset normalizado y lo desnormalizamos para guardarlo todo junto en RIAK

In [3]:
df_mov = pd.read_excel("../../data/black.xlsx", sheet_name= "Movimientos")
df_miembros = pd.read_excel("../../data/black.xlsx", sheet_name= "Miembros")
df = pd.merge(df_mov, df_miembros, on = ['id_miembro'], how = 'inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7624 entries, 0 to 7623
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id_miembro          7624 non-null   int64         
 1   fecha               7624 non-null   datetime64[ns]
 2   minuto              7624 non-null   int64         
 3   hora                7624 non-null   int64         
 4   importe             7624 non-null   float64       
 5   comercio            6882 non-null   object        
 6   actividad_completa  7621 non-null   object        
 7   actividad           7621 non-null   object        
 8   nombre              7624 non-null   object        
 9   funcion             7624 non-null   object        
 10  organizacion        6086 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 714.8+ KB


In [4]:
df.head(5)

,id_miembro,fecha,minuto,hora,importe,comercio,actividad_completa,actividad,nombre,funcion,organizacion
0,12,2008-12-28,18,16,37.70,BEISALFLOR,RESTAURANTES RESTO,RESTAURANTE,Cándido Cerón Escudero,concejal,Partido Popular
1,12,2010-09-07,17,16,83.00,EL ASADOR DE ARANDA S A,RESTAURANTES RESTO,RESTAURANTE,Cándido Cerón Escudero,concejal,Partido Popular
2,12,2008-10-31,41,15,127.80,CASA RIOS,RESTAURANTES RESTO,RESTAURANTE,Cándido Cerón Escudero,concejal,Partido Popular
3,12,2008-12-08,26,16,116.59,CASA RIOS,RESTAURANTES RESTO,RESTAURANTE,Cándido Cerón Escudero,concejal,Partido Popular
4,12,2009-03-06,26,21,22.28,INDUSTRIAS HOSTELERIA JUNCO,RESTAURANTES RESTO,RESTAURANTE,Cándido Cerón Escudero,concejal,Partido Popular


## Riak ...

In [5]:
# connect to database
cliente = riak.RiakClient()
cliente.ping()

True

Utilizamos un bucket para guardar los movimientos en formato JSON y otro para guardar el importe acumulado por miembro.

En el bucket de movimientos utilizamos una clave autogenerada (no vamos a poder localizar los movimientos por clave) y en el bucket con el importe acumulado la clave es el nombre de la persona en formato HASH, para no tener problemas ni con los espacios del nombre ni con los acentos

Se van a crear los siguientes buckets:

|bucket|Clave|Contenido|
|-|-|-|
|movimientos|Marca temporal|Todos los datos del dataset en formnato JSON y desnormalizados|
|acum_importes|Código HASH de la persona que realiza el movimiento|Mapa con el nombre de la persona y el importe acumulado|

El bucket de tweets tendrá los siguientes índices:

|Nombre del índice|Contenido|¿Que busquedas permite hacer?|
|-|-|-|
|idx_miembro_bin|Nombre de la persona|Localizar movimientos de una persona concreta|

In [6]:
BUCKET_MOVIMIENTOS = 'movimientos'
BUCKET_ACUM_IMPORTES = 'acum_importes'

bucket_mov = cliente.bucket(BUCKET_MOVIMIENTOS)
bucket_acum_importes = cliente.bucket_type('maps').bucket(BUCKET_ACUM_IMPORTES)

Función para eliminar los datos de un BUCKET de Riak

In [7]:
def drop_keys(bucket):
    for keys in bucket.stream_keys():
        for key in keys:
            bucket.delete(key)

In [8]:
# Borramos los datos ..
drop_keys(bucket_mov)
drop_keys(bucket_acum_importes)

## Inserción de información en Riak

Para generar los datos en formato JSON partimos del DataFrame de Pandas y lo exportamos a JSON (en formato String) para luego cargarlo en un diccionario de Python

Observa que cuando se genera el dato en formato JSON, los campos fechas se almacenan como un [TIMESTAMP de unix](http://www.unixtimestamp.com), por lo que habría que volver a convertirlo a fecha según el caso

In [9]:
json_string = df.to_json(orient = 'records')
json_list = json.loads(json_string)

In [10]:
print(len(json_list))

7624


In [11]:
pp(json_list[0])

{
    'actividad': 'RESTAURANTE',
    'actividad_completa': 'RESTAURANTES RESTO',
    'comercio': 'BEISALFLOR',
    'fecha': 1230422400000,
    'funcion': 'concejal',
    'hora': 16,
    'id_miembro': 12,
    'importe': 37.7,
    'minuto': 18,
    'nombre': 'Cándido Cerón Escudero',
    'organizacion': 'Partido Popular',
}


Vamos a utilizar el nombre de la persona como clave, por lo que necesitamos convertirla previamente a un código que no lleve ni espacios ni acentos.

Este es precisamente el objetivo de la siguiente función ...

In [12]:
def hash_string(s):
    s_utf8 = s.encode('utf-8')
    return hashlib.md5(s_utf8).hexdigest()

Recorremos el dataset de movimientos, almacenando la información tanto en el bucket de movimientos como en el bucket que acumula los movimientos por cliente.

En este último utilizamos una funcionalidad que nos da Riak para guardar la información tipificada. Utilizamos un mapa que tiene dos elementos: Un registro donde guardamos el nombre del cliente y un contador donde guardamos el importe (sin decimales)

In [15]:
# Bucle de carga de datos ...
for movimiento_json in json_list:
    key = uuid.uuid1().hex
    hash_nombre =  hash_string(movimiento_json["nombre"])
    
    movimiento = bucket_mov.new(key, movimiento_json)
    movimiento.add_index('idx_miembro_bin', hash_nombre)
    movimiento.store()
    
    map_nombre = bucket_acum_importes.new(hash_nombre)
    map_nombre.registers['nombre'].assign(movimiento_json["nombre"])
    map_nombre.counters['importe'].increment(int(round(movimiento_json["importe"] * 100)))
    map_nombre.store()
    

# Lectura de información en Riak

En la siguiente función partimos del objeto JSON que nos devuelve RIAK y generamos un DataFrame Pandas.

En el caso de la fecha, lo convertimos desde un UNIX Timestamp al formato correcto para Pandas

In [16]:
def json_to_pandas(rows):
    # Convertimos el objeto JSON en un objeto pandas 
    df = pd.read_json(json.dumps(rows))
    
    if 'fecha' in df.columns:
        # Las fechas están en formato UNIX TIMESTAMP. Las volvemos a convertir a formato Date...
        df = df.assign(fecha = pd.to_datetime(df.fecha, unit = 'ms'))

    return df

### Lectura del bucket de movimientos

Cargamos todos los movimientos de la base de datos y lo guardamos en un objeto Pandas, ya que hay ciertas preguntas que no pueden resolverse directamente por esta base de datos

Recuerda que RIAK permite obtener información de un clave, pero no le es posible devolver la información ordenada ...

In [17]:
rows = []
for keys in bucket_mov.stream_keys():
    for key in keys:
        # print('Key %s' % key )
        rows.append(bucket_mov.get(key).data)
        
# Convertimos el objeto json en un objeto pandas 
df_movimientos = json_to_pandas(rows)

In [18]:
df_movimientos.head()

,id_miembro,fecha,minuto,hora,importe,comercio,actividad_completa,actividad,nombre,funcion,organizacion
0,40,2007-08-04,19,16,159.97,BOCALEON IBERICA SL,RESTAURANTES RESTO,RESTAURANTE,José María Arteta Vico,concejal,PSOE
1,40,2006-02-03,15,16,98.00,THE CAPI TAVERN,"CAFETERIAS,SNACKS",RESTAURANTE,José María Arteta Vico,concejal,PSOE
2,40,2005-11-04,55,16,94.34,EL PALADAR DE YIL -SCHASTIE-SL,RESTAURANTES RESTO,RESTAURANTE,José María Arteta Vico,concejal,PSOE
3,40,2008-09-19,29,16,60.13,PIZZERIA NABUCCO,RESTAURANTES RESTO,RESTAURANTE,José María Arteta Vico,concejal,PSOE
4,19,2003-10-07,32,15,40.00,GUIROPA S A,GASOLINERAS,COCHE,Francisco José Moure Bourio,concejal,Partido Popular


### Los 10 movimientos mas caros

Esta consulta no la podemos contestar directamente con la Base de Datos por lo que nos apoyamos en un proceso en el cliente

In [19]:
df_movimientos.sort_values('importe', ascending=False)[['nombre', 'fecha', 'actividad_completa', 'importe']].head(10)

,nombre,fecha,actividad_completa,importe
1043,Carmen Contreras Gómez,2010-08-15,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",9076.76
1816,Carmen Contreras Gómez,2010-08-15,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",9076.76
3159,Ildefonso José Sánchez Barcoj,2010-05-19,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),7500.00
7672,Ildefonso José Sánchez Barcoj,2010-05-19,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),7500.00
3252,Ildefonso José Sánchez Barcoj,2008-11-19,EL CORTE INGLES,6593.20
2879,Ildefonso José Sánchez Barcoj,2008-11-19,EL CORTE INGLES,6593.20
8473,Miguel Blesa de la Parra,2006-12-26,EL CORTE INGLES,6397.31
281,Miguel Blesa de la Parra,2006-12-26,EL CORTE INGLES,6397.31
1988,Ramón Ferraz Ricarte,2005-11-27,EL CORTE INGLES,6248.00
4965,Ramón Ferraz Ricarte,2005-11-27,EL CORTE INGLES,6248.00


### Lectura a través de un índice

In [20]:
rows = []
keys = bucket_mov.stream_index("idx_miembro_bin", hash_string(u"Javier de Miguel Sánchez"))
for keys in keys.results:
    for movimiento_key in keys:
        rows.append(bucket_mov.get(movimiento_key).data)
    
df = json_to_pandas(rows)

### Los movimientos de una persona concreta

Para obtener los movimientos de una persona en concreto si podemos utilizar el índice que habíamos creado Ad-hoc, aunque la ordenación se realiza en Pandas

In [21]:
df.sort_values('importe', ascending=False)[['nombre', 'fecha', 'actividad_completa', 'importe']].head(10)

,nombre,fecha,actividad_completa,importe
73,Javier de Miguel Sánchez,2003-02-11,EL CORTE INGLES,1190.00
94,Javier de Miguel Sánchez,2009-05-22,"MATERIALES CONSTRUCCION,FONTANERIA,SANEA",380.00
124,Javier de Miguel Sánchez,2010-01-12,"MATERIALES CONSTRUCCION,FONTANERIA,SANEA",345.00
37,Javier de Miguel Sánchez,2006-03-10,VETERINARIA (VENTA Y CURA DE ANIMALES),332.02
104,Javier de Miguel Sánchez,2009-12-22,"MATERIALES CONSTRUCCION,FONTANERIA,SANEA",272.85
137,Javier de Miguel Sánchez,2007-03-27,SERV.ORG.PUBL.Y GRANDES EMPRESAS,269.92
36,Javier de Miguel Sánchez,2008-07-09,EL CORTE INGLES,262.39
119,Javier de Miguel Sánchez,2007-08-24,EL CORTE INGLES,260.00
91,Javier de Miguel Sánchez,2010-01-12,"MATERIALES CONSTRUCCION,FONTANERIA,SANEA",253.02
163,Javier de Miguel Sánchez,2005-09-05,HIPERCOR SUPERMERCADOS EL CORTE INGLES,243.97


### Lectura de una información agregada

La información agregada está en un mapa, por lo que tenemos que procesarla y convertirla a formato JSON.

In [22]:
rows = []
for keys in bucket_acum_importes.stream_keys():
    #print(keys)
    for key in keys:
        map_nombre = bucket_acum_importes.get(key)
        rows.append({'nombre' : map_nombre.registers['nombre'].value,
                     "importe" : float(map_nombre.counters['importe'].value) / 100})
        
# Convertimos el objeto json en un objeto pandas 
df = json_to_pandas(rows)

### Las 10 personas que mas han gastado

Fácil si disponemos de un agregado. Utilizamos Pandas para ordenar la información ..,

In [23]:
df.sort_values('importe', ascending=False).head(10)

,nombre,importe
62,Ramón Ferraz Ricarte,92965.08
39,Ricardo Morado Iglesias,91709.74
78,Ildefonso José Sánchez Barcoj,85101.30
70,Miguel Blesa de la Parra,76595.54
20,Carmen Contreras Gómez,67811.08
40,Antonio Romero Lázaro,58254.04
16,Rubén Cruz Orive,43519.92
54,José Antonio Moral Santín,42184.51
2,José María de la Riva Amez,41728.18
46,Carlos María Martínez Martínez,40985.22
